In [3]:
import numpy as np
import pandas as pd 
import random 
import math

class City():
    def __init__(self,name,x,y):
        self.name = name
        self.x = x
        self.y = y
        
    def distance(self,city): #the formula for the euclidien distance
        xDistance = (city.x - self.x)**2
        yDistance = (city.y - self.y)**2
        return np.sqrt(abs(xDistance + yDistance))
    
    def getName(self):
        return self.name
    
    def getCoordinates(self):        
        return self.x,self.y
    
class Route():
    def __init__(self,cities):
        self.cities = cities
        self.cost = self.calculateCost()
        
    def calculateCost(self):
        cost = 0.0
        for index in range(len(self.cities)-1):
            if index == len(self.cities)-1:#if We reach the end of the circuit we go back to the first step 
                nextIndex = 0
            else:
                nextIndex = index + 1 #else we go to the next city  
            cost = cost + self.cities[index].distance(self.cities[nextIndex])
        return cost   

    def  getCost(self):
        return self.cost
    
    def getNames(self):
        names = []
        for element in self.cities:
            names.append(element.getName())
        return names
    
    def setFitness(self):
        self.fitness = 1 / self.cost
            
class Tsp():
    def __init__(self,cities):
        self.cities = cities
        self.distances = self.distances_toDF()
        
    def distances_toDF(self):
        self.names = []
        data = {}

        for city in self.cities :#set the header of the dataframe that contains all the cities in their order
            self.names.append(city.name)
            
        defeaultValues = []
        for index in range(len(self.names)):#to initialize the values of  columns 
            defeaultValues.append(0)
            
        for name in self.names:#fill the columns with their default values
            data[name] = defeaultValues
            
        distancesDF = pd.DataFrame(data,self.names)#the actual df
        
        for i in range(len(self.cities)-1):#calculate the actual distance between each two cities 
            city1 = self.cities[i]
            for j in range(i+1,len(self.cities)):
                c2 = self.cities[j]
                distance = city1.distance(c2)
                distancesDF.loc[city1.name][c2.name] = distance
                distancesDF.loc[c2.name][city1.name] = distance
                distance = 0.0
        return distancesDF
    
    def getCities(self):
            return self.cities
        
    def getDistances(self):
            return self.distances 
        
    def glouton1(self):#using the nearest neighbor
        startingIndex =  random.randint(0, len(self.cities)-1)#pick a random city to start 
        rows = self.names #get the header
        currentIndex = startingIndex 
        workingRow = []
        sortedWorkingRow = []
        passedCities = [self.cities[startingIndex]]#keep track of the traveled cities 
       # return workingRow
        for i in range(len(self.cities)):      
            workingRow = self.distances[rows[currentIndex]].tolist()#transform the df row into a list 
            sortedWorkingRow = self.distances[rows[currentIndex]].tolist()
            sortedWorkingRow.sort()
            for currentSpot in sortedWorkingRow: #for every element in the row we count it's occurence in case we have different vetecies 
                occurences = [i for i, x in enumerate(workingRow) if x == currentSpot] #with the same values
                for j in occurences:
                    currentIndex = j
                    if not self.cities[currentIndex] in passedCities:#check if we have already passed by the edge 
                        passedCities.append(self.cities[currentIndex])
        passedCities.append(self.cities[startingIndex])#return to the starting point 
        return passedCities
    
    def glouton2(self):#same as glouton1 but we specify the starting vertex as the samllest value different then 0
        minList = []
        rows = self.names #get the header
        workingRow = []
        sortedWorkingRow = []
        for element in range(len(self.cities)):
            minList.append(self.distances[rows[element]].tolist()[0])
        minList.sort()
        startingIndex = minList[0] #pick optimal city to start 
        currentIndex = startingIndex 
        passedCities = [self.cities[startingIndex]]#keep track of the traveled cities
        for i in range(len(self.cities)):      
            workingRow = self.distances[rows[currentIndex]].tolist()#transform the df row into a list 
            sortedWorkingRow = self.distances[rows[currentIndex]].tolist()
            sortedWorkingRow.sort()
            for currentSpot in sortedWorkingRow: #for every element in the row we count it's occurence in case we have different vetecies 
                occurences = [i for i, x in enumerate(workingRow) if x == currentSpot] #with the same values
                for j in occurences:
                    currentIndex = j
                    if not self.cities[currentIndex] in passedCities:#check if we have already passed by the edge 
                        passedCities.append(self.cities[currentIndex])
        passedCities.append(self.cities[startingIndex])#return to the starting point 
        return passedCities
    
    def twoOptSwap(self,route,i,j):
        part1 = route[:i-1]
        part2 = route[i-1:j]
        part2.reverse()
        part3 = route[j:]
        for element in part2:
            part1.append(element)
        for element in part3:
            part1.append(element)
        return part1
    
    def calculateCost(self,cityList):
        cost = 0.0
        for index in range(len(cityList)-1):
            if index == len(cityList)-1:#if We reach the end of the circuit we go back to the first step 
                nextIndex = 0
            else:
                nextIndex = index + 1 #else we go to the next city  
            cost = cost + cityList[index].distance(cityList[nextIndex])
            
        return cost
    def descente(self):
        #picking the starting solution : go through the cities by the given order in glouton1 
        #startingSolution = []
        #for element in self.cities : 
        #    startingSolution.append(element)
        solution = self.glouton2()
        #generate a space of neighbor solutions 
        SpaceOfSolutions = []
        optimalSolution = solution
        counter = 0 
        while counter < 20:
            for element in range(5):#get 5 solutions
                SpaceOfSolutions.append(self.twoOptSwap(optimalSolution,element%len(solution),element+4 %len(solution)))
                if self.calculateCost(optimalSolution) < self.calculateCost(SpaceOfSolutions[element]):
                    return optimalSolution
                optimalSolution = element
                counter = counter + 1 
        return optimalSolution
    def P(self,s,s1,T):
        return  math.exp(-(s-s1)/T)
        
    def recuitSimule(self):
        solution = self.glouton2()#the starting solution from glouton2
        SpaceOfSolutions = []
        T = 500
        counter = 0 
        while counter < 20 : 
            for i in range(5):
                SpaceOfSolutions.append(self.twoOptSwap(solution.copy(),i % len(solution),i+4 % len(solution)))
            r = random.random()
            for index in range(5):
                if r < self.P(self.calculateCost(solution),self.calculateCost(SpaceOfSolutions[index]),T):
                        return solution
                solution = SpaceOfSolutions[index]
                T = T - 20
                counter = counter + 1
        return solution
    
    def Tabou(self):
        solution = self.glouton1()
        tabou = [self.twoOptSwap(solution.copy(),4 % len(solution),7 % len(solution))]
        counter = 0
        optimalSolution = solution 
        while counter < 20 :
                i = random.randint(1,len(solution))
                j = random.randint(i,len(solution))
                tempSolution = self.twoOptSwap(solution.copy(),i,j)
                if not tempSolution in tabou :
                    tabou.append(tempSolution)
                    if self.calculateCost(optimalSolution) > self.calculateCost(tempSolution):
                        optimalSolution = tempSolution
                counter = counter + 1 
        return optimalSolution
    
    def reverse_segment_if_better(self,tour, i, j, k):
        """If reversing tour[i:j] would make the tour shorter, then do it."""
        # Given tour [...A-B...C-D...E-F...]
        A, B, C, D, E, F = tour[i-1], tour[i], tour[j-1], tour[j], tour[k-1], tour[k % len(tour)]
        d0 = self.distance(A, B) + self.distance(C, D) + self.distance(E, F)
        d1 = self.distance(A, C) + self.distance(B, D) + self.distance(E, F)
        d2 = self.distance(A, B) + self.distance(C, E) + self.distance(D, F)
        d3 = self.distance(A, D) + self.distance(E, B) + self.distance(C, F)
        d4 = self.distance(F, B) + self.distance(C, D) + self.distance(E, A)

        if d0 > d1:
            tour[i:j] = reversed(tour[i:j])
            return -d0 + d1
        elif d0 > d2:
            tour[j:k] = reversed(tour[j:k])
            return -d0 + d2
        elif d0 > d4:
            tour[i:k] = reversed(tour[i:k])
            return -d0 + d4
        elif d0 > d3:
            tmp = tour[j:k] + tour[i:j]
            tour[i:k] = tmp
            return -d0 + d3
        return 0
    def distance(self,a,b):
        costA = Route(a).calculateCost()
        costB = Route(b).calculateCost()
        return costA - costB
    
    def three_opt(self,tour):
        """Iterative improvement based on 3 exchange."""
        while True:
            delta = 0
            for (a, b, c) in self.all_segments(len(tour)):
                delta += self.reverse_segment_if_better(tour, a, b, c)
            if delta >= 0:
                break
        return tour

    def all_segments(self,n: int):
        """Generate all segments combinations"""
        return ((i, j, k)
            for i in range(n)
            for j in range(i + 2, n)
            for k in range(j + 2, n + (i > 0)))
    
    def getChances(self,routes):
        fitness = self.calculateFitness(routes)
        propabilites = []
        for element in fitness:
            propabilites.append(element / sum(fitness))
        return propabilites
    
    def cross(self,parent1,parent2):
        coupure = random.randint(1,len(parent1))
        kid1 = parent1[:coupure-1].append(parent2[coupure-1:])
        kid2 = parent2[:coupure-1].append(parent1[coupure-1:])
        return kid1,kid2
    
    def mutation(self,individual):
        pt1 = 0
        pt2 = 0 
        while pt1 == pt2:
            pt1 = random.randint(0,len(individual))
            pt2 = random.randint(0,len(individual))
        mutated = individual.copy()
        temp = mutated[pt1]
        mutated[pt1] = mutated[pt2]
        mutated[pt2] = temp
        return mutated
        
    def calculateFitness(self,routes):
        fitness = []
        for element in routes:
            fitness.append(1 / self.calculateCost(element))
        return fitness
        
    def randomRoute(self,cities):#random route generation 
        route = random.sample(cities, len(cities))
        return route
    
    def routeFromBinary(self,route,cities):
        newRoute = []
        for element in route: 
            position = 0 
            for i in element:
                if i == 1:
                    newRoute.append(cities[position])
                else:
                    position = position + 1 
        return newRoute
                    
    def initialePopulation(self,cities,pSize):
        population = []
        for element in range(pSize):
            population.append(self.randomRoute(cities))
        return population
    
    def selectPopulation(self,population,selectionType):
        chances = self.getChances(population)
        selectedPopulation = []
        i = 0
        for element in chances:
            if element >= np.mean(chances):
                selectedPopulation.append(population[i])
            i = i + 1
        if selectionType == "cross":#80 %
            return selectedPopulation[:math.floor(len(selectedPopulation)*0.8)]
        elif selectionType == "mutation":#20%
            return selectedPopulation[math.floor(len(selectedPopulation)*0.8):]
        else : 
            return selectedPopulation
            
                    
            
    def randomBinaryRoute(self,cities):#binary coding 
        binaries = []
        row = []
        indices = random.sample(range(len(cities)), len(cities))
        
        for element in indices:
            for i in range(len(indices)):
                if i == element : 
                    row.append(1)
                else:
                    row.append(0)
            binaries.append(row)
            row = []
        
        return binaries
    
    def VNS(self):
        solution = self.glouton1()
        k = 0 
        while k < 4:
            print("")
            
    def getBestRoute(self,routes) : 
        minimum = self.calculateCost(routes[0])
        minIndex  = 0
        index = 0 
        for element in route : 
            cost = self.calculateCost(element)
            if minimum > cost:
                minimum = cost 
                minIndex = index
            index = index + 1 
        return routes[minIndex]
                    
    def genetic(self):
        initialPopulation = self.initialePopulation(self.cities,10)
        t=0
        while t < 10 :
            crossPopulation1 = self.selectPopulation(initialPopulation,"cross")
            mutationPopulation1 = self.selectPopulation(initialPopulation,"mutation")
            crossPopulation2 = []
            for i in range(math.floor(len(crossPopulation1) / 2)) :
                kid1,kid2 = self.cross(crossPopulation1[i],crossPopulation1[len(crossPopulation1)-1 - i])
                crossPopulation2.append(kid1)
                crossPopulation2.append(kid1)
                
            return kid1
        
            mutationPopulation2 = []
            for element in mutationPopulation1:
                mutationPopulation2.append(self.mutation(element))
            
            mutationKids = []
            for i in range(math.floor(len(mutationPopulation2) / 2)) :
                kid1,kid2 = self.cross(mutationPopulation2[i],mutationPopulation2[len(mutationPopulation2) - i])
                mutationKids.append(kid1)
                mutationKids.append(kid2)
            
            initialPopulation = crossPopulation2.append(mutationPopulation2.append(mutationKids))
            t = t + 1
        return self.getBestRoute(initialPopulation)
        


In [2]:
        
city1 = City("a",1,2)
city2 = City("b",2,3)
city3 = City("c",3,5)
city4 = City("d",4,4)
city5 = City("e",6,2)
city6 = City("f",1,3)
tsp = Tsp([city1,city2,city3,city5,city6,city4])
cities = tsp.getCities()
#for element in cities :
 #   print(element.getName())
   # coordinates = element.getCoordinates()
 #   x,y = element.getCoordinates()
 #   print(x)   
  #  print(y)
#route = Route([city1,city3,city2,city4])
#print(route.getCost())
random.randint(0, 10)
df = tsp.distances
#index = df[df['b']== 1].index.values.tolist()
index = df.index
indices = index[df['a'] == 0]#get the list of indices that comforms with the condition 
print(indices.tolist()[0])#get the actual value of the first occurance 
df['c'].tolist()#turn a column into a list 
cities = tsp.glouton1()
route = Route(cities)
print("route cost is : " + str(route.getCost()))
for element in route.getNames():
    print (element + "->")
tsp.twoOptSwap(['a','b','c','d','e','f','g','h','a'],4,7)
cities2 = tsp.descente()
route2 = Route(cities2)
print("route cost is : " + str(route2.getCost()))
for element in route2.getNames():
    print (element + "->")
cities3 = tsp.recuitSimule()
route3 = Route(cities2)
print("route cost is : " + str(route3.getCost()))
for element in route3.getNames():
    print (element + "->")

cities4 = tsp.Tabou()
route4 = Route(cities2)
print("route cost is : " + str(route4.getCost()))
for element in route4.getNames():
    print (element + "->")
tsp.randomRoute(['a','b','c','d','e','f','g','h','a'])
tsp.randomBinaryRoute(['a','b','c','d','e','f','g','h','a'])
tsp.genetic()

a
route cost is : 13.991941740584954
f->
a->
b->
c->
d->
e->
f->
route cost is : 14.485281374238571
a->
b->
f->
c->
d->
e->
a->
route cost is : 14.485281374238571
a->
b->
f->
c->
d->
e->
a->
route cost is : 14.485281374238571
a->
b->
f->
c->
d->
e->
a->
